In [ ]:
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# learning_rate = 1e-4
# keep_prob_rate = 0.7 # 
# max_epoch = 2000
# def compute_accuracy(v_xs, v_ys):
#     global prediction
#     y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
#     correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
#     return result

# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)

# def bias_variable(shape):
#     initial = tf.constant(0.1, shape=shape)
#     return tf.Variable(initial)

# def conv2d(x, W):
#     # 每一维度  滑动步长全部是 1， padding 方式 选择 same
#     # 提示 使用函数  tf.nn.conv2d
#     return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# def max_pool_2x2(x):
#     # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
#     # 提示 使用函数  tf.nn.max_pool
#     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# # define placeholder for inputs to network
# xs = tf.placeholder(tf.float32, [None, 784])/255.
# ys = tf.placeholder(tf.float32, [None, 10])
# keep_prob = tf.placeholder(tf.float32)
# x_image = tf.reshape(xs, [-1, 28, 28, 1])

# #  卷积层 1
# ## conv1 layer ##

# W_conv1 = weight_variable([7, 7, 1, 32])                      # patch 7x7, in size 1, out size 32
# b_conv1 = bias_variable([32])                      
# h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)                      # 卷积  自己选择 选择激活函数
# h_pool1 = max_pool_2x2(h_conv1)                      # 池化               

# # 卷积层 2
# W_conv2 = weight_variable([5, 5, 32, 64])                       # patch 5x5, in size 32, out size 64
# b_conv2 = bias_variable([64])
# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)                       # 卷积  自己选择 选择激活函数
# h_pool2 = max_pool_2x2(h_conv2)                       # 池化

# #  全连接层 1
# ## fc1 layer ##
# W_fc1 = weight_variable([7*7*64, 1024])
# b_fc1 = bias_variable([1024])

# h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# # 全连接层 2
# ## fc2 layer ##
# W_fc2 = weight_variable([1024, 10])
# b_fc2 = bias_variable([10])
# prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# # 交叉熵函数
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
#                                               reduction_indices=[1]))
# train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     for i in range(max_epoch):
#         batch_xs, batch_ys = mnist.train.next_batch(100)
#         sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
#         if i % 100 == 0:
#             print(compute_accuracy(
#                 mnist.test.images[:1000], mnist.test.labels[:1000]))




In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical

# 关闭即时执行模式，兼容 TensorFlow 1.x 代码
tf.compat.v1.disable_eager_execution()


class MnistData:
    def __init__(self):
        (self.train_images, self.train_labels), (
        self.test_images, self.test_labels) = tf.keras.datasets.mnist.load_data()
        self.train_images = self.train_images.reshape(-1, 784) / 255.0
        self.test_images = self.test_images.reshape(-1, 784) / 255.0
        self.train_labels = to_categorical(self.train_labels, num_classes=10)
        self.test_labels = to_categorical(self.test_labels, num_classes=10)

    def next_batch(self, batch_size):
        idx = np.random.choice(len(self.train_images), batch_size)
        return self.train_images[idx], self.train_labels[idx]

mnist = MnistData()

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 2000

xs = tf.compat.v1.placeholder(tf.float32, [None, 784], name='xs')
ys = tf.compat.v1.placeholder(tf.float32, [None, 10], name='ys')
keep_prob = tf.compat.v1.placeholder(tf.float32, name='keep_prob')
x_image = tf.reshape(xs, [-1, 28, 28, 1])


def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool2d(x, ksize=[2, 2], strides=[2, 2], padding='SAME')

# 卷积层 1
W_conv1 = weight_variable([7, 7, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 卷积层 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 全连接层 1
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, rate=1 - keep_prob)

# 全连接层 2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.math.log(prediction), axis=1))


train_step = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1.0})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1.0})
    return result



with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    for i in range(max_epoch):
        batch_xs, batch_ys = mnist.next_batch(100)
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: keep_prob_rate})

        if i % 100 == 0:
            acc = compute_accuracy(mnist.test_images[:1000], mnist.test_labels[:1000])
            loss = sess.run(cross_entropy, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 1.0})
            print(f"Step {i}, Accuracy: {acc:.4f}, Loss: {loss:.4f}")




Step 0, Accuracy: 0.1480, Loss: 4.5476
Step 100, Accuracy: 0.8510, Loss: 0.4730
Step 200, Accuracy: 0.9020, Loss: 0.3733
Step 300, Accuracy: 0.9290, Loss: 0.2754
Step 400, Accuracy: 0.9430, Loss: 0.1857
Step 500, Accuracy: 0.9550, Loss: 0.1529
Step 600, Accuracy: 0.9630, Loss: 0.1480
Step 700, Accuracy: 0.9690, Loss: 0.1672
Step 800, Accuracy: 0.9670, Loss: 0.2344
Step 900, Accuracy: 0.9690, Loss: 0.0610
Step 1000, Accuracy: 0.9720, Loss: 0.0984
Step 1100, Accuracy: 0.9730, Loss: 0.0484
Step 1200, Accuracy: 0.9740, Loss: 0.0340
Step 1300, Accuracy: 0.9780, Loss: 0.0442
Step 1400, Accuracy: 0.9760, Loss: 0.1211
Step 1500, Accuracy: 0.9810, Loss: 0.0498
Step 1600, Accuracy: 0.9800, Loss: 0.0422
Step 1700, Accuracy: 0.9800, Loss: 0.0346
Step 1800, Accuracy: 0.9840, Loss: 0.0227
Step 1900, Accuracy: 0.9810, Loss: 0.0109
